In [1]:
import librosa
import numpy as np
from scipy.fftpack import dct
import os 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import wave
import math
import scipy.io.wavfile as wav
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier

Split wave

Load Data

In [2]:
def load_data(data_dir):
    real_files = [os.path.join(data_dir, "real", f) for f in os.listdir(os.path.join(data_dir, "real")) if f.endswith(".wav")]
    fake_files = [os.path.join(data_dir, "fake", f) for f in os.listdir(os.path.join(data_dir, "fake")) if f.endswith(".wav")]
    print(len(real_files))
    print(len(fake_files))
    fake_labels = [0] * len(fake_files)
    real_labels = [1] * len(real_files)

    files = fake_files + real_files
    labels = fake_labels + real_labels

    return files, labels

Preprocessing

In [3]:
def evaluate(frames):
    # Calculate the average energy of the frames
    frame_energies = np.sum(np.square(frames), axis=1)
    avg_energy = np.mean(frame_energies)

    return avg_energy

In [4]:
def optimal_fft_length(window_length: int) -> int:
    return 2 ** int(np.ceil(np.log2(window_length)))

In [5]:
def preprocessing(audio_file,n_features):
    # Load the audio signal
    y, sr = librosa.load(audio_file, sr=None)

    # Step 1: Pre-emphasis
    pre_emphasis_coeff = 0.97
    y_filt = librosa.effects.preemphasis(y, coef=pre_emphasis_coeff)

    # Step 2: Frame blocking
    frame_length = 0.02  # 20 ms
    hop_length = 0.010    # 10 ms
    frame_length_samples = int(frame_length * sr)
    hop_length_samples = int(hop_length * sr)
    frames = librosa.util.frame(y_filt, frame_length= frame_length_samples, hop_length= hop_length_samples)

    # Step 3: Windowing

    frame_length = len(frames)
    output = frames*np.hamming(frame_length)[:, np.newaxis]
    frames = output

    # Step 4: Fast Fourier Transform (FFT)
    fft_size = optimal_fft_length(len(frames))
    spectrogram = np.abs(np.fft.fft(frames, n=fft_size))
    
    # Step 5: Mel frequency wrapping
    n_mels = n_features
    mel_spec = librosa.feature.melspectrogram(sr=sr, S=spectrogram, n_mels=n_mels)

    # Step 6: Discrete Cosine Transform (DCT)
    n_mfcc = 216
    mfcc = dct(np.log(mel_spec), type=2, axis=1, norm='ortho')[:, :n_mfcc]

    # Scaled MFCC features
    max_mfcc=np.max(mfcc, axis=1)
    mins, maxs=np.min(max_mfcc), np.max(max_mfcc)
    scaled_mfcc=(max_mfcc-mins)/(maxs-mins)

    return scaled_mfcc

Train the model

Preparing Data

In [7]:
url = r"C:\Users\VIET HOANG - VTS\Desktop\tien xu ly"
files, labels = load_data(url)

X_train, X_test, y_train, y_test = train_test_split(files, labels, test_size=0.2, random_state=42)

X_train = [preprocessing(file,68) for file in X_train]
X_test = [preprocessing(file,68) for file in X_test]

941
661


In [42]:
model = KNeighborsClassifier(n_neighbors=5, p=2, weights='uniform')
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 62.93%


In [38]:
model = RandomForestClassifier()

# Define the hyperparameters grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required at each leaf node
}

# Perform grid search with cross-validation
model= GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=5)
model.fit(X_train, y_train)

# Modelin doğruluğunu değerlendirme
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 68.54%


In [39]:
model.best_params_

{'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 10,
 'n_estimators': 200}

In [ ]:
print(len(files))

196


In [32]:
from sklearn import svm
# Create an SVM classifier
# Define the SVM model
svc = svm.SVC()

# Set up the parameter grid
param_grid = {
    'C': [0.1, 1, 10,20,100],                 # Penalty parameter C
    'kernel': ['rbf'],        # Kernel function: linear or radial basis function (rbf)
    'gamma': ['scale', 'auto', 0.1, 1, 0.01]  # Kernel coefficient for 'rbf'. 'scale' and 'auto' use default values.
}

# Perform grid search with cross-validation
clf= GridSearchCV(estimator=svc, param_grid=param_grid, cv=5)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

clf.best_params_

Test Accuracy: 71.03%


{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}

In [36]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# Define the hyperparameters grid for tuning
param_grid = {
    'penalty': ['l1', 'l2'],  # Regularization penalty
    'C': [0.1, 1.0, 10.0],  # Inverse of regularization strength
    'solver': ['liblinear', 'saga','lbfgs','saga'],  # Solver algorithm
    'max_iter' : [10,20,50,100]
}

# Perform grid search with cross-validation
clf = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=5)
clf.fit(X_train,y_train)
# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed 

Test Accuracy: 65.11%


c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "

In [37]:
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

Test Accuracy: 65.11%


In [8]:
rf = RandomForestClassifier(max_depth=None, min_samples_leaf=2, min_samples_split=2, n_estimators=200)
svmm = svm.SVC(C=10, gamma=0.1, kernel='rbf', probability=True)

# # Define the voting classifier
# voting_clf = VotingClassifier(
#     estimators=[('svm', svmm), ('rf', rf)],
#     voting='soft'
# )
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.1, 0.5, 1.0]
}

# Define the AdaBoost classifier with the voting classifier as the base estimator
adaboost_clf = AdaBoostClassifier(estimator=svmm, n_estimators=50, learning_rate=0.1)

grid_search = GridSearchCV(adaboost_clf, param_grid=param_grid)
# Fit the AdaBoost classifier
grid_search.fit(X_train, y_train)

# Calculate the accuracy score
accuracy = grid_search.score(X_test, y_test)
print("Accuracy: ", accuracy)

c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\VIET HOANG - VTS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarn

In [55]:
print(len(X_test))

321
